In [1]:
import pandas as pd
import numpy as np
import pickle
import joblib
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.svm import SVR

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/TA

Mounted at /content/drive
/content/drive/MyDrive/TA


In [2]:
# === Load Dataset ===
df =  pd.read_csv('Train.csv')

# Memisahkan fitur (X) dan target (y)
X = df.drop(columns=["Pic50"])
y = df["Pic50"]

# === Load Fitur ===
with open('selected_feature_rbf.pkl', 'rb') as file:
    featuredrbf = pickle.load(file)

In [3]:
# Filter data
x_trainrbf = X[featuredrbf]
x_trainrbf

,PubchemFP2,PubchemFP8,PubchemFP10,PubchemFP13,PubchemFP15,PubchemFP16,PubchemFP22,PubchemFP23,PubchemFP26,PubchemFP27,...,PubchemFP861,PubchemFP862,PubchemFP863,PubchemFP864,PubchemFP866,PubchemFP868,PubchemFP871,PubchemFP874,PubchemFP876,PubchemFP879
0,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
192,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
193,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
194,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y_trainrbf = y.ravel()

<ipython-input-4-1903ee7e1ef6>:1: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_trainrbf = y.ravel()


In [5]:
y_trainrbf

array([7.        , 7.48148606, 9.45593196, 9.15490196, 6.39637373,
       8.11350928, 5.82390874, 4.69897   , 9.15490196, 5.92081875,
       4.8951716 , 8.06550155, 7.79588002, 4.65757732, 5.537602  ,
       5.537602  , 9.22184875, 6.32513886, 9.74713203, 7.52287874,
       6.49894074, 6.45222529, 6.37675071, 9.33724217, 3.60205999,
       6.22184875, 8.00877392, 6.87614836, 7.33724217, 7.01772877,
       6.75696195, 6.3419886 , 4.46533942, 8.63827216, 8.50863831,
       6.91009489, 8.79588002, 3.82390874, 6.20971484, 7.76955108,
       8.37675071, 6.22914799, 6.66154351, 8.50863831, 6.85387196,
       7.93181414, 6.46852108, 8.79588002, 7.58502665, 6.45345734,
       9.44533171, 8.43179828, 7.20065945, 9.39794001, 6.06550155,
       8.39794001, 6.01772877, 7.69897   , 6.39794001, 5.25963731,
       7.1739252 , 8.82390874, 9.25963731, 7.88605665, 6.        ,
       3.84163751, 7.76955108, 6.1079054 , 8.65757732, 8.18708664,
       6.        , 7.22914799, 8.49485002, 6.76955108, 7.39794

In [6]:
# === Parameter Tuning dengan GridSearchCV ===
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'gamma': ['scale', 'auto'],
              'kernel': ['rbf']
}

grid_rbf = GridSearchCV(SVR(), param_grid=param_grid, refit=True, cv=10, scoring='r2', verbose= 3)
grid_rbf.fit(x_trainrbf, y_trainrbf)

print('Parameter Terbaik rbf', grid_rbf.best_params_)

# === Evaluasi Model ===
# cross_val_score untuk menghitung rata-rata skor R2
score_rbf = cross_val_score(grid_rbf.best_estimator_, x_trainrbf, y_trainrbf, cv=10, scoring='r2')
score_rbf = np.average(score_rbf)

print('Skor rbf: ',score_rbf)

Fitting 10 folds for each of 14 candidates, totalling 140 fits
[CV 1/10] END C=0.001, gamma=scale, kernel=rbf;, score=-0.003 total time=   0.0s
[CV 2/10] END C=0.001, gamma=scale, kernel=rbf;, score=-0.116 total time=   0.0s
[CV 3/10] END C=0.001, gamma=scale, kernel=rbf;, score=-0.015 total time=   0.0s
[CV 4/10] END C=0.001, gamma=scale, kernel=rbf;, score=-0.069 total time=   0.0s
[CV 5/10] END C=0.001, gamma=scale, kernel=rbf;, score=-0.202 total time=   0.0s
[CV 6/10] END C=0.001, gamma=scale, kernel=rbf;, score=-0.245 total time=   0.0s
[CV 7/10] END C=0.001, gamma=scale, kernel=rbf;, score=-0.069 total time=   0.0s
[CV 8/10] END C=0.001, gamma=scale, kernel=rbf;, score=-0.007 total time=   0.0s
[CV 9/10] END C=0.001, gamma=scale, kernel=rbf;, score=-0.197 total time=   0.0s
[CV 10/10] END C=0.001, gamma=scale, kernel=rbf;, score=-0.042 total time=   0.0s
[CV 1/10] END .C=0.001, gamma=auto, kernel=rbf;, score=-0.005 total time=   0.0s
[CV 2/10] END .C=0.001, gamma=auto, kernel=rb

In [7]:
# Save File
joblib.dump(grid_rbf.best_estimator_,"grid_rbf.best_estimator.pkl")

['grid_rbf.best_estimator.pkl']

In [8]:
grid_rbf.cv_results_

{'mean_fit_time': array([0.01410086, 0.01321447, 0.0127784 , 0.01352899, 0.01440096,
        0.0132    , 0.01270795, 0.01352396, 0.01546798, 0.01325724,
        0.01771824, 0.01595953, 0.01905637, 0.03045959]),
 'std_fit_time': array([0.00214546, 0.00133168, 0.00043995, 0.00130275, 0.00301348,
        0.00237903, 0.00053743, 0.00201133, 0.00176153, 0.00104825,
        0.00111509, 0.00121586, 0.0009059 , 0.00410316]),
 'mean_score_time': array([0.00716434, 0.00727775, 0.00728564, 0.00764041, 0.00783477,
        0.00718362, 0.00673597, 0.00679998, 0.00747848, 0.00774102,
        0.00693698, 0.00744047, 0.00750737, 0.00741956]),
 'std_score_time': array([0.00025392, 0.00057291, 0.000601  , 0.00088023, 0.00152871,
        0.00098334, 0.00042884, 0.00030293, 0.0013314 , 0.00261923,
        0.0006294 , 0.00123422, 0.00118884, 0.00079015]),
 'param_C': masked_array(data=[0.001, 0.001, 0.01, 0.01, 0.1, 0.1, 1.0, 1.0, 10.0,
                    10.0, 100.0, 100.0, 1000.0, 1000.0],
              

In [9]:
# === Hasil GridSearchCV ===
results = grid_rbf.cv_results_

for params, mean_test_score in zip(results['params'], results['mean_test_score']):
    print(f"Parameter: {params} -> Skor Mean Test: {mean_test_score:.4f}")
    print()

# for params, mean_test_score in zip(results['params'], results['mean_test_score']):
#   print("Parameter:", params)
#   print("Skor Mean Test:", mean_test_score)
#   print()

Parameter: {'C': 0.001, 'gamma': 'scale', 'kernel': 'rbf'} -> Skor Mean Test: -0.0965

Parameter: {'C': 0.001, 'gamma': 'auto', 'kernel': 'rbf'} -> Skor Mean Test: -0.1008

Parameter: {'C': 0.01, 'gamma': 'scale', 'kernel': 'rbf'} -> Skor Mean Test: -0.0488

Parameter: {'C': 0.01, 'gamma': 'auto', 'kernel': 'rbf'} -> Skor Mean Test: -0.0938

Parameter: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'} -> Skor Mean Test: 0.2024

Parameter: {'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'} -> Skor Mean Test: -0.0223

Parameter: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'} -> Skor Mean Test: 0.5145

Parameter: {'C': 1, 'gamma': 'auto', 'kernel': 'rbf'} -> Skor Mean Test: 0.2686

Parameter: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'} -> Skor Mean Test: 0.5847

Parameter: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'} -> Skor Mean Test: 0.5233

Parameter: {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'} -> Skor Mean Test: 0.4995

Parameter: {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'} -> Skor Mean Test: 0

In [10]:
# Evaluasi Model Parameter default untuk RBF
param_default = {
    'C': 1,
    'gamma': 'scale',
    'kernel': 'rbf'}

model_rbf = SVR(**param_default)

# model_rbf = SVR(
#     C=param_default['C'],
#     gamma=param_default['gamma'],
#     kernel=param_default['kernel']
# )

#Evaluasi
score_rbf_after = cross_val_score(model_rbf, x_trainrbf, y_trainrbf, cv=10, scoring='r2')
score_rbf_after = np.average(score_rbf_after)
print("R2 Score Default RBF:", score_rbf_after)

R2 Score Default RBF: 0.514469247345555


In [11]:
# Evaluasi Model Parameter terbaik untuk RBF
param_terbaik =  {
    'C': 10,
    'gamma': 'scale',
    'kernel': 'rbf'}

model_rbf = SVR(**param_terbaik)

# model_rbf = SVR(
#     C=param_terbaik['C'],
#     gamma=param_terbaik['gamma'],
#     kernel=param_terbaik['kernel']
# )

#Evaluasi
score_rbf_after = cross_val_score(model_rbf, x_trainrbf, y_trainrbf, cv=10, scoring='r2')
score_rbf_after = np.average(score_rbf_after)
print("R2 Score Best Model RBF:", score_rbf_after)


R2 Score Best Model RBF: 0.5846589803716148
